# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,olonkinbyen,70.9221,-8.7187,-2.65,53,67,6.28,SJ,1705632615
1,1,albany,42.6001,-73.9662,-5.71,87,98,1.06,US,1705632390
2,2,port-aux-francais,-49.3500,70.2167,7.42,97,100,11.37,TF,1705632616
3,3,edinburgh of the seven seas,-37.0676,-12.3116,16.15,62,16,10.71,SH,1705632617
4,4,kerikeri,-35.2268,173.9474,27.73,79,85,1.79,NZ,1705632618


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.8,
    alpha = 0.5,
    color = "City",
    frame_width = 700,
    frame_height = 500,
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# max temperature < 27 degrees but > 21
# wind speed < 4.5 m/s
# cloudiness = 0

city_ideal_weather_df = city_data_df[(city_data_df['Max Temp']<27) & (city_data_df['Max Temp']>21) & (city_data_df['Wind Speed']<45) & (city_data_df['Cloudiness'] == 0)].dropna()
city_ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,carnarvon,-24.8667,113.6333,26.94,53,0,10.39,AU,1705632633
94,94,ceelbuur,4.6850,46.6176,22.25,84,0,5.86,SO,1705632682
99,99,crucecita,-28.9500,-58.9833,23.07,90,0,3.36,AR,1705632685
110,110,wailua homesteads,22.0669,-159.3780,26.44,62,0,6.17,US,1705632693
137,137,bubaque,11.2833,-15.8333,26.01,71,0,7.18,GW,1705632712
177,177,aquin,18.2833,-73.4000,24.48,84,0,1.41,HT,1705632739
257,257,ceeldheer,3.8488,47.1806,24.59,79,0,7.28,SO,1705632794
260,260,lihue,21.9789,-159.3672,26.77,61,0,6.17,US,1705632797
349,349,seme,15.1942,-12.9448,24.99,12,0,4.71,SN,1705632861
355,355,salalah,17.0151,54.0924,25.05,29,0,5.14,OM,1705632866


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_weather_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,carnarvon,AU,-24.8667,113.6333,53,
94,ceelbuur,SO,4.6850,46.6176,84,
99,crucecita,AR,-28.9500,-58.9833,90,
110,wailua homesteads,US,22.0669,-159.3780,62,
137,bubaque,GW,11.2833,-15.8333,71,
177,aquin,HT,18.2833,-73.4000,84,
257,ceeldheer,SO,3.8488,47.1806,79,
260,lihue,US,21.9789,-159.3672,61,
349,seme,SN,15.1942,-12.9448,12,
355,salalah,OM,17.0151,54.0924,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set the geographical coordinates 
latitude = 40.415392
longitude = -3.7073743182788528

# Set the parameters for the type of search
limit = 5
radius = 10000
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"

# set up a parameters dictionary
params = {
    "filter":filters,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key,
    "categories":f"accommodation.hotel"
}

#params["categories"] = f"accommodation.hotel"

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

hotel = requests.get(base_url, params=params).json()
hotel

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Hotel Pestana Plaza Mayor',
    'country': 'Spain',
    'country_code': 'es',
    'state': 'Community of Madrid',
    'city': 'Madrid',
    'postcode': '28012',
    'district': 'Centro',
    'neighbourhood': 'Barrio de los Austrias',
    'suburb': 'Sol',
    'street': 'Calle Imperial',
    'housenumber': '8',
    'lon': -3.7073011,
    'lat': 40.4150128,
    'formatted': 'Hotel Pestana Plaza Mayor, Calle Imperial, 8, 28012 Madrid, Spain',
    'address_line1': 'Hotel Pestana Plaza Mayor',
    'address_line2': 'Calle Imperial, 8, 28012 Madrid, Spain',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': ['details', 'details.contact'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Hotel Pestana Plaza Mayor'

In [7]:
city_hotel = hotel["features"][0]["properties"]["name"]
city_hotel

'Hotel Pestana Plaza Mayor'

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":f"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_hotel = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_hotel = name_hotel.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_hotel["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carnarvon - nearest hotel: No hotel found
ceelbuur - nearest hotel: No hotel found
crucecita - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
bubaque - nearest hotel: Chez Julio
aquin - nearest hotel: Hotel Aldy
ceeldheer - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
seme - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
pisco - nearest hotel: La Portada
kulhudhuffushi - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
eyl - nearest hotel: فندق ايل دابي
bull savanna - nearest hotel: Junction Guest House
piacabucu - nearest hotel: O Leão
ugoofaaru - nearest hotel: No hotel found
lafiagi - nearest hotel: No hotel found
dinguiraye - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms


,City,Country,Lat,Lng,Humidity,Hotel Name
25,carnarvon,AU,-24.8667,113.6333,53,No hotel found
94,ceelbuur,SO,4.6850,46.6176,84,No hotel found
99,crucecita,AR,-28.9500,-58.9833,90,No hotel found
110,wailua homesteads,US,22.0669,-159.3780,62,Hilton Garden Inn Kauai Wailua Bay
137,bubaque,GW,11.2833,-15.8333,71,Chez Julio
177,aquin,HT,18.2833,-73.4000,84,Hotel Aldy
257,ceeldheer,SO,3.8488,47.1806,79,No hotel found
260,lihue,US,21.9789,-159.3672,61,Kauai Palms
349,seme,SN,15.1942,-12.9448,12,No hotel found
355,salalah,OM,17.0151,54.0924,29,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.8,
    alpha = 0.5,
    color = "City",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)